In [30]:
import tensorflow as tf
import numpy as np


In [31]:
# 1. Define Network
def base_model():
    inputs = tf.keras.layers.Input(shape=(784,), name="clothing")
    x = tf.keras.layers.Dense(64, activation = 'relu', name = 'dense_1')(inputs)
    x = tf.keras.layers.Dense(64, activation = 'relu', name='dense_2')(x)
    outputs = tf.keras.layers.Dense(10, activation = 'softmax', name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = base_model()

In [32]:
import tensorflow_datasets as tfds

# 2. Prepare Training Data pipeline

train_data = tfds.load("fashion_mnist", split = 'train')
test_data = tfds.load("fashion_mnist", split = 'test')
def format_image(data):
    image = data['image']
    image = tf.reshape(image, [-1])
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.0
    return image, data['label']

train_data = train_data.map(format_image)
test_date = test_data.map(format_image)

batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test = test_data.batch(batch_size)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [33]:
# 3. Define Loss and Optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


In [34]:
# Define Metrics
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [35]:
# 4. Define Custom Training Loop
def apply_gradient(optimizer, model, x, y):

    with tf.GradientTape() as t:
        logits = model(x)
        loss_value = loss_object(y_true=y, y_pred=logits)

    gradients = t.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradient(zip(gradients, model.trainable_weights))
    
    return logits, loss_value



In [36]:
from tqdm import tqdm

def train_data_for_one_epoch():
    losses = []
    pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}' )

    for step, (x_batch_train, y_batch_train) in enumerate(train): 
        logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
        losses.append(loss_value)
        train_acc_metric(y_batch_train, logits)
        pbar.set_description("Training loss for step %s : %.4f" %(int(step), float(loss_value)))
        pbar.update()
    return losses


In [37]:
# Metrics in Keras
## Metrics can be modelled as Class and function
## mean_squared_error(...) / class MeanSquaredError

In [38]:
# Low level handling of Metrics
## metric.update_state() to accumulate metric statistics after each batch
## metric.result to get current value of metric for display
## metric.reset_state() to reset metric value typically at end of epoch

In [39]:
def perform_validation():
    losses = []
    for x_val, y_val in test:
        val_logits = model(x_val)
        val_loss = loss_object(y_val, val_logits)
        losses.append(val_loss)
        val_acc_metric(y_val, val_logits)
    return losses

In [40]:
model = base_model()

epochs = 10
epochs_val_losses, epochs_train_losses = [], []

for epoch in range(epochs):
    losses = train_data_for_one_epoch()
    train_acc = train_acc_metric.result()

    val_losses = perform_validation()
    val_acc = val_acc_metric.result()
    
    losses_train_mean = np.mean(losses)
    losses_val_mean = np.mean(val_losses)

    epochs_train_losses.append(losses_train_mean)
    epochs_val_losses.append(losses_val_mean)

    print(epoch, float(losses_train_mean, losses_val_mean, train_acc, val_acc))
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()
    

2022-01-06 16:27:03.883465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-06 16:27:03.908544: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz


TypeError: 'module' object is not callable